In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd, numpy as np
if 'sim' not in globals():
    import os; os.chdir('..')
import models
import urbansim.sim.simulation as sim
from spandex import TableLoader

loader = TableLoader()

In [31]:
#Nonres
b = sim.get_table('buildings')
msa_id = b.msa_id
b = b.to_frame(b.local_columns)
b['msa_id'] = msa_id
new_nrsf_msa = b[(b.year_built > 2005) & (b.development_type_id.isin([2,3,4,5]))].groupby('msa_id').non_residential_sqft.sum()
nonres_targets = new_nrsf_msa/new_nrsf_msa.sum()
print nonres_targets

msa_id
0         0.068475
1         0.268959
2         0.209235
3         0.046141
4         0.222778
5         0.184189
6         0.000223
Name: non_residential_sqft, dtype: float64


In [56]:
#RES LUZ target
p = sim.get_table('parcels')
p = p.to_frame(columns=['luz_id', 'mgra_id', 'msa_id'])
target_mgras = np.unique(p[p.luz_id.isin([69, 70, 72])].mgra_id)
res_estimates = pd.read_csv(loader.get_path('calibration_data/estimates_du_by_yr_2004_to_2010.csv'))
res_estimates['du'] = res_estimates.hs_sf + res_estimates.hs_sfmu + res_estimates.hs_mf
res_estimates = res_estimates.query('year in [2004, 2010]')
region_units = res_estimates.groupby('year').du.sum()
res_estimates = res_estimates[res_estimates.mgra.isin(target_mgras)]
target_units = res_estimates.groupby('year').du.sum()
target_diff = target_units.loc[2010] - target_units.loc[2004]
region_diff = region_units.loc[2010] - region_units.loc[2004]
target_proportion = target_diff*1.0/region_diff
target_proportion = np.round(target_proportion, decimals = 2)
print target_proportion

0.01


In [ ]:
#Validation
p = sim.get_table('parcels')
p = p.to_frame(columns=['luz_id', 'mgra_id', 'msa_id'])
target_mgras = np.unique(p[p.luz_id.isin([69, 70, 72])].mgra_id)
res_estimates = pd.read_csv(loader.get_path('calibration_data/estimates_du_by_yr_2004_to_2010.csv'))
res_estimates['du'] = res_estimates.hs_sf + res_estimates.hs_sfmu + res_estimates.hs_mf
res_estimates = res_estimates.query('year in [2004, 2010]')
region_units = res_estimates.groupby('year').du.sum()
res_estimates = res_estimates[res_estimates.mgra.isin(target_mgras)]
target_units = res_estimates.groupby('year').du.sum()
target_diff = target_units.loc[2010] - target_units.loc[2004]
region_diff = region_units.loc[2010] - region_units.loc[2004]
target_proportion = target_diff*1.0/region_diff
target_proportion = np.round(target_proportion, decimals = 2)
print target_proportion

In [24]:
#Residential validation target
p = sim.get_table('parcels')
p = p.to_frame(columns=['luz_id', 'mgra_id', 'msa_id'])
res_estimates = pd.read_csv(loader.get_path('calibration_data/estimates_residential_units_households.csv'))
mgra_msa_xref = p.groupby(['mgra_id', 'msa_id']).size().reset_index()
res_estimates10 = res_estimates[res_estimates.estimates_year == 2010]
res_estimates14 = res_estimates[res_estimates.estimates_year == 2014]
merged10 = pd.merge(res_estimates10, mgra_msa_xref, left_on = 'mgra', right_on = 'mgra_id')
merged14 = pd.merge(res_estimates14, mgra_msa_xref, left_on = 'mgra', right_on = 'mgra_id')
du_msa_2010 = merged10.groupby('msa_id').Total_residential_units.sum()
du_msa_2014 = merged14.groupby('msa_id').Total_residential_units.sum()
diffs = du_msa_2014 - du_msa_2010
proportion_growth_captured = diffs/diffs.sum()

In [27]:
#Nonres validation

b = sim.get_table('buildings')
msa_id = b.msa_id
b = b.to_frame(b.local_columns)
b['msa_id'] = msa_id
new_nrsf_msa = b[(b.year_built > 2009) & (b.development_type_id.isin([2,3,4,5]))].groupby('msa_id').non_residential_sqft.sum()
nonres_validation = new_nrsf_msa/new_nrsf_msa.sum()
print nonres_validation

msa_id
0         0.343273
1         0.115041
2         0.030555
3         0.020157
4         0.199530
5         0.291049
6         0.000394
Name: non_residential_sqft, dtype: float64
